In [1]:
import os, math, random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
%matplotlib inline

from time import sleep
import keras.backend as K

# from keras.backend.tensorflow_backend import set_session
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True 
# config.log_device_placement = True 
# sess = tf.Session(config=config)
# set_session(sess)


def read_tfrecord(example):
    features = {
        'image': tf.FixedLenFeature([], tf.string),
        'labels': tf.FixedLenFeature([], tf.string)
    }
    sample=tf.parse_single_example(example, features)
    image = tf.decode_raw(sample['image'], tf.uint8)
    image = tf.reshape(image, tf.stack([224, 224, 3]))
    image = augmentation(image)
    labels = tf.decode_raw(sample['labels'], tf.float64)
    labels = tf.reshape(labels, tf.stack([2,19]))
    labels = labels[:,:15] #remove head joints
    labels = tf.cast(labels, tf.float32)
    return image, labels/224.

def load_dataset(filenames):
    files = tf.data.Dataset.list_files(filenames)
    dataset = files.apply(tf.data.experimental.parallel_interleave(tf.data.TFRecordDataset, cycle_length=4))
    dataset = dataset.apply(tf.data.experimental.map_and_batch(map_func=read_tfrecord, batch_size=BATCH_SIZE, drop_remainder=True))
    dataset = dataset.apply(tf.data.experimental.shuffle_and_repeat(128, -1))
    dataset = dataset.prefetch(buffer_size=128)
    return dataset

def augmentation(img):
    image = tf.cast(img, tf.float32)
    image = tf.image.random_brightness(image, max_delta=25/255)
    image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
    image = tf.image.random_contrast(image, lower=0.5, upper=1.5)
    image = tf.image.per_image_standardization(image)
    return image

def get_batched_dataset(filenames):
    dataset = load_dataset(filenames)
    return dataset

def get_training_dataset():
    return get_batched_dataset(training_filenames)

def get_validation_dataset():
    return get_batched_dataset(validation_filenames)

Using TensorFlow backend.


In [2]:
def create_model(f=32):
    
    
    def bn_rl_conv(x, f, k=1, s=1):
        x = layers.BatchNormalization()(x)
        x = tf.keras.layers.ReLU(max_value=1. , negative_slope= 0 ,threshold= 0)(x)
        x = layers.Conv2D(f, k, strides=s, padding=p)(x)
        return x

    def dense_block(tensor, r):
        for _ in range(r):
            x = bn_rl_conv(tensor, 4*f)
            x = bn_rl_conv(x, f, 3)
            tensor = layers.Concatenate()([tensor, x])
        return tensor

    def transition_block(x):
        x = bn_rl_conv(x, K.int_shape(x)[-1] // 2)
        x = layers.MaxPool2D(2, strides=2)(x)
        return x
    
    
    repetitions = 6, 12, 24, 16
    
    Input = layers.Input(shape=(224, 224, 3))
    x = layers.Conv2D(64, 7, strides=2)(Input)
    x = layers.MaxPool2D(3, strides=2)(x)
    
    
    for r in repetitions:
        d = dense_block(x, r)
        x = transition_block(d)
        
    
    x = layers.Conv2D( 30, 3)(x)
#     x = layers.Conv2D( 30, 3, strides=2)(x)
    x = tf.keras.layers.ReLU(max_value=1. , negative_slope= 0 ,threshold= 0)(x)
    
    
    x = tf.keras.layers.Reshape((2,15))(x)
    model = tf.keras.Model(inputs=Input, outputs=x)
    model.compile(optimizer=keras.optimizers.Adam(0.01), loss='mean_squared_error')
    return model

model=create_model()

model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 56, 56, 64)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 56, 56, 64)   256         max_pooling2d[0][0]              
_____________

In [3]:
EPOCHS=50
name='model_dense2'
BATCH_SIZE = 16
bucket_train = '/home/antreas/Desktop/tf_records_file/try/*'
bucket_valid = '/home/antreas/Desktop/tf_records_file/try/*'
train = tf.gfile.Glob(bucket_train)
valid = tf.gfile.Glob(bucket_valid)
training_filenames = train
validation_filenames = valid
steps_per_epoch = int(len(train)*1000/(BATCH_SIZE)) ###### to 1000 einai to plithos ton eikonon sto tfrecord #######
val_steps = int(len(valid)*1000/(BATCH_SIZE))


######## Tensorboard ############
logdir='/home/antreas/Desktop/model_feb/'+name
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir, write_graph=True)
######## Model Save #############
filepath = '/home/antreas/Desktop/model_feb/'+name+'.hdf5'
checkpoint = keras.callbacks.ModelCheckpoint((filepath),monitor='val_loss', save_best_only=True, verbose=1, save_weights_only=False)
#################################
clbk=[tensorboard_callback, checkpoint]

model=create_model()

model.fit(get_training_dataset(), validation_data=get_validation_dataset(), initial_epoch=0,
          steps_per_epoch=steps_per_epoch ,
          validation_steps=val_steps, epochs=EPOCHS, verbose=1, callbacks=clbk)


Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.
Instructions for updating:
Use `tf.data.Dataset.map(map_func, num_parallel_calls)` followed by `tf.data.Dataset.batch(batch_size, drop_remainder)`. Static tf.data optimizations will take care of using the fused implementation.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use `tf.data.Dataset.shuffle(buffer_size, seed)` followed by `tf.data.Dataset.repeat(count)`. Static tf.data optimizations will take care of using the fused implementation.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 62 steps, validate on 62 steps
Epoch 1/50
61/62 [============================>.] - ETA: 0s - loss: 0.1458
Epoch 00001: val_loss improved from inf to 0.1

Epoch 28/50
61/62 [============================>.] - ETA: 0s - loss: 0.1423
Epoch 00028: val_loss did not improve from 0.14269
62/62 [==============================] - 24s 393ms/step - loss: 0.1427 - val_loss: 0.1427
Epoch 29/50
61/62 [============================>.] - ETA: 0s - loss: 0.1423
Epoch 00029: val_loss did not improve from 0.14269
62/62 [==============================] - 24s 393ms/step - loss: 0.1427 - val_loss: 0.1427
Epoch 30/50
61/62 [============================>.] - ETA: 0s - loss: 0.1423
Epoch 00030: val_loss did not improve from 0.14269
62/62 [==============================] - 24s 394ms/step - loss: 0.1427 - val_loss: 0.1427
Epoch 31/50
61/62 [============================>.] - ETA: 0s - loss: 0.1428
Epoch 00031: val_loss did not improve from 0.14269
62/62 [==============================] - 24s 393ms/step - loss: 0.1427 - val_loss: 0.1427
Epoch 32/50
61/62 [============================>.] - ETA: 0s - loss: 0.1422
Epoch 00032: val_loss did not improve from 0.14269
62/62

In [2]:
model = tf.keras.models.load_model('/home/antreas/Desktop/model_feb/model_dense2.hdf5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [3]:
train_images=np.load('/home/antreas/Desktop/test_images/RGB_IM_ARRAY.npy')##### Images

y=train_images[25]
im = np.expand_dims(y, axis=0)
x = model.predict(im)

x[0]

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0.]],
      dtype=float32)